In [ ]:
# import os
# os.getcwd()
#!pip install dotmap

In [1]:
import sys

# make sure to use position 1
# sys.path.insert(1, "/jet/prs/workspace/Keras-Project-Template-Jupyter")
sys.path.insert(1, "/Users/mtavako1/Documents/Research/__Radiation_Therapy/Code/1_KerasLearning/Keras-Project-Template-Jupyter")

In [2]:
from utils.config import process_config
from utils.dirs import create_dirs
from utils.utils import get_args

from data_loader.prostate_dist_dvh_data_loader import ProstateDistDvhDataLoader
from models.transfer_3block_vgg16_model import Transfer3BlockVGG16Model
from models.transfer_4block_rnn_vgg16_model import Transfer4BlockRnnVGG16Model

from trainers.simple_trainer import SimpleTrainer

import h5py
import numpy as np
from sklearn.model_selection import StratifiedKFold


Using TensorFlow backend.


In [3]:
def main(config_file):
    # capture the config path from the run arguments
    # then process the json configuration file
    try:
        config = process_config(config_file)
        #print config
    except:
        assert False, "missing or invalid arguments"
        
    # create the experiments dirs
    print('Create the experiments dirs.')
    create_dirs([config.callbacks.tensorboard_log_dir, config.callbacks.checkpoint_dir])

    print('Create the data generator.')
    data_loader = ProstateDistDvhDataLoader(config)
    
    
    # cross validation
    cvscores = []
    kfold = StratifiedKFold(
        n_splits=config.cross_validation.n_splits,
        shuffle=config.cross_validation.shuffle,
        random_state=config.cross_validation.random_state
    )
    
    for train, test in kfold.split(data_loader.get_data()[0], data_loader.get_data()[1]):
        
        # print('Create the model.')
        model = Transfer4BlockRnnVGG16Model(config)
        #Transfer4BlockRnnVGG16Model(config)
        
        # train or train-val 
        # print('Create the trainer')
        trainer = SimpleTrainer(model.model, data_loader.get_data(), config)

        print('Start training/validating the model.')
        # trainer.train()
        scores = trainer.train_val(train, test)
    
        for metric, score in zip(model.model.metrics_names, scores):
            print("%s: %.2f%%" % (metric, score*100))

        cvscores.append(scores)

    means = np.mean(cvscores, axis=0)
    stds = np.std(cvscores, axis=0)

    for metric, mean, std in zip(model.model.metrics_names, means, stds):
        print("%s: %.2f%% (+/- %.2f%%)" % (metric, mean * 100, std * 100))

if __name__ == '__main__':
    main("configs/vgg16_4blocks_rnn_from_config.json")    
    # main("configs/vgg16_4blocks_rnn_from_config.json")

Create the experiments dirs.
Create the data generator.
Start training/validating the model.
Train on 128 samples, validate on 15 samples
Epoch 1/2
128/128 [==============================] - 64s 498ms/step - loss: 0.6971 - precision: 0.2781 - recall: 0.5039 - fbeta_score: 0.3517 - acc: 0.4766 - val_loss: 0.6970 - val_precision: 0.4667 - val_recall: 1.0000 - val_fbeta_score: 0.6200 - val_acc: 0.4667
Epoch 2/2
73/73 [==============================] - 16s 222ms/step
loss: 69.27%
precision: 52.05%
recall: 100.00%
fbeta_score: 68.23%
acc: 52.05%
Start training/validating the model.
Train on 129 samples, validate on 15 samples
Epoch 1/2
129/129 [==============================] - 69s 532ms/step - loss: 0.6855 - precision: 0.5271 - recall: 0.4550 - fbeta_score: 0.4372 - acc: 0.5969 - val_loss: 0.6328 - val_precision: 0.9000 - val_recall: 0.7000 - val_fbeta_score: 0.7429 - val_acc: 0.8000
Epoch 2/2
72/72 [==============================] - 18s 246ms/step
loss: 67.26%
precision: 64.76%
recall: 31

In [ ]:
#!pip install dotmap
# !pip install comet_ml
#!pip install --upgrade tensorflow tensorflow-tensorboard
#!pip install comet_ml
# !pip install tensorflow==1.8.0

In [ ]:
#!tensorboard --logdir="/Users/mtavako1/Documents/Research/__Radiation_Therapy/Code/1_KerasLearning/Keras-Project-Template-Jupyter/experiments/2018-10-29/vgg16_3blocks_from_config/"

In [ ]:
# get_config_from_json("configs/simple_mnist_config.json")

In [ ]:
# https://github.com/keunwoochoi/keras_callbacks_example
# GAN: https://www.youtube.com/watch?v=0VPQHbMvGzg

In [ ]:
tuple([1]) + (3,4)


In [ ]:
from keras.applications.vgg16 import VGG16 
from keras.models import Model 
from keras.layers import Dense, Input 
from keras.layers.pooling import GlobalAveragePooling2D 
from keras.layers.recurrent import LSTM 
from keras.layers.wrappers import TimeDistributed 
from keras.optimizers import Nadam

video = Input(shape=(frames, channels, rows, columns)) 
cnn_base = VGG16(input_shape=(channels, rows, columns), weights="imagenet", include_top=False) 
cnn_out = GlobalAveragePooling2D()(cnn_base.output) 
cnn = Model(input=cnn_base.input, output=cnn_out) 
cnn.trainable = False 

encoded_frames = TimeDistributed(cnn)(video) 
encoded_sequence = LSTM(256)(encoded_frames) 
hidden_layer = Dense(output_dim=1024, activation="relu")(encoded_sequence) 
outputs = Dense(output_dim=classes, activation="softmax")(hidden_layer) 
model = Model([video], outputs) 
optimizer = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004) 
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["categorical_accuracy"])
